In [13]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [14]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Naryan-Mar,RU,2021-02-05 23:47:05,67.6713,53.0870,-12.89,89,39,11.03
1,1,Naryan-Mar,RU,2021-02-05 23:47:05,67.6713,53.0870,-12.89,89,39,11.03
2,2,Ushuaia,AR,2021-02-05 23:55:09,-54.8000,-68.3000,46.40,76,75,25.32
3,3,Busselton,AU,2021-02-05 23:57:29,-33.6500,115.3333,61.00,76,95,4.00
4,4,Avarua,CK,2021-02-05 23:57:29,-21.2078,-159.7750,86.00,70,20,13.80


In [15]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [18]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [24]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [32]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [35]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Avarua,CK,2021-02-05 23:57:29,-21.2078,-159.7750,86.00,70,20,13.80
8,8,Butaritari,KI,2021-02-05 23:57:29,3.0707,172.7902,81.70,80,81,19.01
9,9,Rikitea,PF,2021-02-05 23:57:30,-23.1203,-134.9692,79.07,69,5,12.66
13,13,Natitingou,BJ,2021-02-05 23:57:30,10.3042,1.3796,77.09,18,33,5.77
20,20,Conceicao Do Araguaia,BR,2021-02-05 23:57:31,-8.2578,-49.2647,77.14,88,100,3.42
21,21,Carnarvon,AU,2021-02-05 23:54:19,-24.8667,113.6333,80.60,100,90,24.16
24,24,Vaini,TO,2021-02-05 23:57:32,-21.2000,-175.2000,84.20,84,75,6.91
29,29,Lakes Entrance,AU,2021-02-05 23:57:33,-37.8811,147.9810,75.00,53,1,14.00
38,38,Progreso,MX,2021-02-05 23:57:34,21.2833,-89.6667,89.60,37,1,9.22
43,43,Paita,PE,2021-02-05 23:57:35,-5.0892,-81.1144,75.40,73,17,17.05


In [36]:
preferred_cities_df.count()

City_ID       316
City          316
Country       315
Date          316
Lat           316
Lng           316
Max Temp      316
Humidity      316
Cloudiness    316
Wind Speed    316
dtype: int64

In [37]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Avarua,CK,86.00,-21.2078,-159.7750,
8,Butaritari,KI,81.70,3.0707,172.7902,
9,Rikitea,PF,79.07,-23.1203,-134.9692,
13,Natitingou,BJ,77.09,10.3042,1.3796,
20,Conceicao Do Araguaia,BR,77.14,-8.2578,-49.2647,
21,Carnarvon,AU,80.60,-24.8667,113.6333,
24,Vaini,TO,84.20,-21.2000,-175.2000,
29,Lakes Entrance,AU,75.00,-37.8811,147.9810,
38,Progreso,MX,89.60,21.2833,-89.6667,
43,Paita,PE,75.40,-5.0892,-81.1144,


In [38]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}